**Assignment - 7**

**Team Members:**

1. Sanjeeth Busnur Indushekar: 224133 : sanjeeth.busnur@st.ovgu.de

2. Aditya Dey : 230580 : aditya.dey@st.ovgu.de

3. Suraj Shashidhar: 230052 : suraj.shashidhar@st.ovgu.de

In [0]:
import tensorflow as tf
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

In [0]:
os.chdir('/content/drive/My Drive/Colab Notebooks')

# Task-1

In [0]:
@tf.function
def top_k_values(Input,k):
  top_k = tf.math.top_k(input=Input,k=k,sorted=False)
  return top_k.values

In [0]:
Input = tf.reshape(np.arange(25),[5,5])
Input.numpy()

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [0]:
tf.print(top_k_values(Input,3))

[[2 3 4]
 [7 8 9]
 [12 13 14]
 [17 18 19]
 [22 23 24]]


# Task-2

In [0]:
@tf.function
def one_hot_argmax(Input):
  argmax = tf.math.argmax(input=Input,axis=1)
  return tf.one_hot(argmax,depth=Input.shape[1])

In [0]:
Input = tf.random.uniform((6,4),minval=1,maxval=25,dtype=tf.dtypes.int32)
Input.numpy()

array([[ 3, 11, 15, 14],
       [ 7,  9, 14,  3],
       [ 4,  5, 23, 20],
       [14,  1,  1, 13],
       [ 7, 20, 12,  4],
       [16, 16,  6, 11]], dtype=int32)

In [0]:
tf.print(one_hot_argmax(Input))

[[0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 0 0]]


# Task-3

**One Dimensional Array**

In [0]:
@tf.function
def scatter(Input,k):
  new_arr = tf.math.top_k(Input,k=k)
  indices = tf.expand_dims(new_arr.indices,axis=1)
  scatter = tf.scatter_nd(indices=indices,updates=new_arr.values,shape=Input.shape)
  return scatter

In [0]:
Input = tf.random.uniform((5,),minval=1,maxval=25,dtype=tf.dtypes.int32)
tf.print(Input)

[1 3 9 23 14]


In [0]:
tf.print(scatter(Input,3))

[0 0 9 23 14]


**Two Dimensional Array**

In [0]:
@tf.function
def scatter_2d(Input, k):
  shape = Input.shape
  new_arr = tf.math.top_k(Input,k=k)
  indices = new_arr.indices
  values = new_arr.values
  num_rows = Input.shape[0]
  row_range = tf.range(num_rows)
  tensor_row = tf.tile(row_range[:,None], (1, k))
  top_k_row_col_idxs = tf.stack([tensor_row, indices], axis=2)
  updates = tf.ones([num_rows, k], dtype=tf.int32)
  zero_mask = tf.scatter_nd(top_k_row_col_idxs, updates, shape)
  scatter = Input * zero_mask
  return scatter

In [0]:
Input = tf.random.uniform((5,5),minval=1,maxval=50,dtype=tf.dtypes.int32)
tf.print(Input)

[[17 1 12 19 32]
 [25 4 40 44 40]
 [37 36 18 40 39]
 [21 4 11 40 21]
 [36 29 5 37 11]]


In [0]:
tf.print(scatter_2d(Input, 3))

[[17 0 0 19 32]
 [0 0 40 44 40]
 [37 0 0 40 39]
 [21 0 0 40 21]
 [36 29 0 37 0]]


Reference:
1. https://stackoverflow.com/questions/52996505/using-scatter-nd-with-top-k-output

# Task-4

In [0]:
@tf.function
def exp_mov_avg(Input,a):
  new = tf.TensorArray(tf.float32, size=0, dynamic_size=True,clear_after_read=False)
  new = new.write(0,Input[0])
  for i in tf.range(1,Input.shape[0]):
    new = new.write(i,(a*new.read(i-1))+((1-a)*Input[i]))
  return new.stack()
  

In [0]:
Input = tf.constant([1,2,3,4,5],dtype=float)
a = tf.constant(0.15,dtype=float)

In [0]:
tf.print(exp_mov_avg(Input,a))

[1 1.85 2.8275 3.82412505 4.82361889]


# Task-5

In [0]:
# 5th task didnt know to find it for nth term.
def exp_mov_avg_last(input, a):

    n = int(-100. / np.log(a)) # Makes sure that the first and last elements in f are very big and very small (about 1e22 and 1e-22)
    f = np.exp(np.arange(1-n, n, 2) * (0.5 * np.log(a))) # Scaling factor for each slice
    tmp = (np.resize(input, ((len(input) + n - 1) // n, n)) / f * (1. - a)).cumsum(axis=1) * f # Get ewm for each slice of length n

    # Add the last value of each previous slice to the next slice with corresponding scaling factor f and return result
    return np.resize(tmp + np.tensordot(np.append(input[0], np.roll(tmp.T[n-1], 1)[1:]), f * ((a) / f[0]), axes=0), len(input))

In [21]:
input = [1.,2.,3.,4.,5.,6.]
a = 0.20
tf.print("================= Input before calculating Exponentially moving Average =================== \n")
tf.print(input)
tf.print("================= Last Output after calculating Exponentially moving Average =================== \n")
output = task5(input,a)
tf.print(output[-1])

================= Input before calculating Exponentially moving Average =================== 

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
================= Last Output after calculating Exponentially moving Average =================== 

5.750080000000003


Reference: https://stackoverflow.com/questions/42869495/numpy-version-of-exponential-weighted-moving-average-equivalent-to-pandas-ewm 

# Task - 6

In [0]:
@tf.function
def even_odd(x,y,z):
  original_shape = x.shape
  x = tf.unstack(tf.reshape(x, [-1]))
  y = tf.unstack(tf.reshape(y, [-1]))
  z = tf.unstack(tf.reshape(z, [-1]))
  new_arr = tf.TensorArray(dtype=tf.int32,size=0,dynamic_size=True,clear_after_read=False)
  for i,m in enumerate(x):
    if (m % 2 == 0):
      new_arr = new_arr.write(i,y[i])  
    else:
       new_arr = new_arr.write(i,z[i]) 
  return (tf.reshape(new_arr.stack(),shape=original_shape))
  

In [0]:
x = tf.random.uniform((2,3,3),minval=0,maxval=100,dtype=tf.dtypes.int32)
y = tf.random.uniform((2,3,3),minval=0,maxval=100,dtype=tf.dtypes.int32)
z = tf.random.uniform((2,3,3),minval=0,maxval=100,dtype=tf.dtypes.int32)
tf.print('x: ', x)
tf.print('\n')
tf.print('y: ', y)
tf.print('\n')
tf.print('z: ', z)
tf.print('\n')

x:  [[[37 97 99]
  [94 28 81]
  [53 16 83]]

 [[15 32 45]
  [10 1 3]
  [60 41 21]]]


y:  [[[48 88 77]
  [8 45 86]
  [81 78 31]]

 [[84 87 79]
  [79 97 16]
  [59 81 79]]]


z:  [[[8 61 42]
  [91 20 78]
  [53 70 11]]

 [[32 32 25]
  [67 39 86]
  [54 18 70]]]




In [0]:
tf.print(even_odd(x,y,z))

[[[8 61 42]
  [8 45 78]
  [53 78 11]]

 [[32 87 25]
  [79 39 86]
  [59 18 70]]]


# Task-7

In [0]:
@tf.function
def last_dimension(Input):
  last_dims = Input.shape[-1]
  if (last_dims > 100):
    return 100
  elif (last_dims <= 100 and last_dims > 44):
    return 12
  else:
    return 0

In [0]:
Input = tf.random.uniform(shape=(4,6,15,45),minval=1,maxval=1000)
tf.print(last_dimension(Input))

12


# Task-8

In [0]:
c1 = tf.Variable(0)
c2 = tf.Variable(0)
c3 = tf.Variable(0)

In [0]:
@tf.function
def last_dimension(Input):
  global c1,c2,c3
  last_dims = Input.shape[-1]
  if (last_dims > 100):
    c1.assign_add(1)
    return 100
  elif (last_dims <= 100 and last_dims > 44):
    c2.assign_add(1)
    return 12
  else:
    c3.assign_add(1)
    return 0

In [0]:
Input = tf.random.uniform(shape=(3,2,145),minval=1,maxval=100)
for m in tf.range(5):
   last_dimension(Input)
tf.print(c1,c2,c3)

Input = tf.random.uniform(shape=(4,5,77),minval=1,maxval=100)
for m in tf.range(15):
   last_dimension(Input)
tf.print(c1,c2,c3)

Input = tf.random.uniform(shape=(7,8,7),minval=1,maxval=100)
for m in tf.range(10):
   last_dimension(Input)
tf.print(c1,c2,c3)

5 0 0
5 15 0
5 15 10


# Task-9

In [0]:
@tf.function
def n_n_Tensor(x,y):
  return x[:,None]-y[None,:]

In [0]:
x = tf.random.uniform((3,),minval=1,maxval=25,dtype=tf.dtypes.int32)
y = tf.random.uniform((3,),minval=1,maxval=25,dtype=tf.dtypes.int32)
tf.print('x: ', x)
tf.print('\n')
tf.print('y: ',y)

x:  [1 14 17]


y:  [19 21 6]


In [0]:
tf.print(n_n_Tensor(x,y))

[[-18 -20 -5]
 [-5 -7 8]
 [-2 -4 11]]


# Task-10

In [7]:
from prepare_data import parse_seq
import pickle
# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")


#data.padded_batch
#batched_data = data.padded_batch(batch_size = 128, drop_remainder=True)
# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))


#batched_categorical_data = data.padded_batch(batch_size=128, padded_shapes=448,padding_values=0, drop_remainder=True)

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}
ch_to_ind = {v: k for k, v in ind_to_ch.items()}
print(vocab)
print(vocab_size)

{'I': 1, 'S': 2, 'G': 3, 'b': 4, 'i': 5, 'r': 6, 'n': 7, 'z': 8, '?': 9, 'j': 10, 'q': 11, '\n': 12, 'h': 13, 'Y': 14, 'l': 15, 'u': 16, ':': 17, 'D': 18, 'p': 19, 'V': 20, ';': 21, '!': 22, 'O': 23, '$': 24, 'B': 25, 'k': 26, 'F': 27, 'X': 28, "'": 29, 'w': 30, 'T': 31, 'M': 32, 'P': 33, 'd': 34, 'e': 35, '.': 36, 'o': 37, 'x': 38, 'm': 39, 'L': 40, 'A': 41, 'W': 42, '[': 43, 'g': 44, 'Q': 45, 'K': 46, ' ': 47, 'U': 48, ',': 49, 'Z': 50, '3': 51, ']': 52, 'H': 53, 'a': 54, 'E': 55, 't': 56, 'c': 57, 'N': 58, 'R': 59, 'J': 60, '&': 61, 's': 62, 'C': 63, 'y': 64, 'f': 65, 'v': 66, '-': 67, '<S>': 0}
68


In [0]:
def onehotencode(ds):
  
  new_data = tf.one_hot(indices = ds, depth = vocab_size)
  return new_data;

new_data = data.map(onehotencode)

In [0]:
batched_vocab_data = data.batch(batch_size=128,drop_remainder=True)
batched_onehotencoded_data = new_data.batch(batch_size=128, drop_remainder=True)

In [11]:
cnt = 0
for element in data:
  cnt = cnt+1
print(cnt)

22981


In [0]:
current_decoder_state_fixed = tf.Variable(tf.keras.initializers.GlorotNormal(seed = 1)(shape=[128,63]))
previous_decoder_state_fixed = tf.Variable(tf.keras.initializers.GlorotNormal(seed = 1)(shape=[128,63]))
current_encoder_state = tf.Variable(tf.keras.initializers.GlorotNormal(seed = 1)(shape=[128,63]))
unscaled_attention_matrix = tf.Variable(tf.keras.initializers.GlorotNormal(seed = 1)(shape=[128,200]))
softmax_attention_matrix = tf.Variable(tf.keras.initializers.GlorotNormal(seed = 1)(shape=[128,200]))

In [13]:
previous_decoder_state_fixed

<tf.Variable 'Variable:0' shape=(128, 63) dtype=float32, numpy=
array([[ 0.01473352, -0.05735376,  0.07342789, ...,  0.09645282,
         0.0628942 ,  0.18157996],
       [-0.01305565, -0.04977272,  0.12497865, ...,  0.05449903,
        -0.18177864, -0.06606366],
       [ 0.01745264, -0.21106663, -0.03510231, ..., -0.12910882,
        -0.07065466, -0.04943841],
       ...,
       [ 0.07512955,  0.02983235, -0.08212686, ..., -0.03503273,
        -0.07772417,  0.12510055],
       [ 0.02088889,  0.19565317,  0.06373518, ...,  0.04682589,
        -0.07369351, -0.19658963],
       [-0.18037029, -0.11600257, -0.01586506, ..., -0.07738629,
         0.03209957,  0.07263009]], dtype=float32)>

In [14]:
time_steps=200
#Pick the 4th timestep decoder state of dimension [128, 1]
for batch_num, (x_batch_train, val_data) in enumerate(zip(batched_onehotencoded_data,batched_vocab_data) ):
  print("===== batch number: {}".format(batch_num))
  for time_step in range(time_steps-1):
    if(time_step == 4):
      previous_decoder_state_fixed = val_data[:, time_step]
    #encoder_state_at_current_time_step = x_batch_train[:, time_step, :]
    #print("shape of each time slice: {} ".format(x_t.shape))
    #lbl_batch = val_data[:, time_step+1]
  break;

===== batch number: 0


In [15]:
previous_decoder_state_fixed

<tf.Tensor: shape=(128,), dtype=int32, numpy=
array([62,  7, 35,  4, 19, 12,  6, 37, 35, 35, 27, 54, 56, 16,  7, 35,  6,
       62, 64, 37, 47,  5, 65, 13, 35, 15, 35, 47,  6, 57, 34, 35, 54, 47,
       56, 32, 35, 56, 47, 47, 35, 13, 14, 47,  7, 47, 62, 47,  5, 55, 56,
       54, 62, 36,  6, 32, 47, 16,  2, 12,  5, 13,  6, 63, 13, 37, 35, 37,
       56,  7, 62,  6, 54,  2, 57, 15, 19, 56, 30,  7, 49, 13, 47, 16, 62,
       35, 37, 16, 47, 12, 21, 41, 12, 56, 15, 39,  7, 49,  1,  1, 16, 64,
       47, 58, 35, 31,  6, 13, 47, 54, 37, 12, 42, 47,  6, 13, 35, 56,  1,
       47, 47, 30, 57, 44, 47, 30, 35, 47], dtype=int32)>

In [16]:
onehotencoded_prev_decoder_state = onehotencode(previous_decoder_state_fixed)
onehotencoded_prev_decoder_state

<tf.Tensor: shape=(128, 68), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [17]:
time_steps=200
output_list = []
for batch_num, (x_batch_train, val_data) in enumerate(zip(batched_onehotencoded_data,batched_vocab_data) ):
  print("===== batch number: {}".format(batch_num))
  for time_step in range(time_steps):
    #(128 * 68) dot product (68 * 128) ==> (128, 128)
    curr_timestep_attention_onehot = tf.matmul(a=x_batch_train[:,time_step,:], b=onehotencoded_prev_decoder_state, transpose_b=True)

    #Printing some stuff for checking
    #print(curr_timestep_attention_onehot)
    #print(tf.math.reduce_sum(curr_timestep_attention_onehot, axis=1, keepdims=False, name=None))

    #Convert the onehot encoded output to categorical for attention
    #(128 * 128) === reduce sum on axis 1 to check attention inside same record, using axis zero would take cross records sum
    # (128 * 128) ===> (128 * 1) ==> one attention value per record per timestep
    output_list.append(tf.math.reduce_sum(curr_timestep_attention_onehot, axis=1, keepdims=False, name=None))
    
  unscaled_outputs = tf.stack(output_list,axis = 1)
  print("====== unscaled thing =============")
  print(unscaled_outputs)
  print()
  print(" ===============. softmax thing ===========")
  softmax_output = tf.nn.softmax(unscaled_outputs, axis=1)
  print(softmax_output)
  print()
  print("======== check whether softmax adds to one for final check, it adds to ne approx(numerical multiplier issues in matmul) ===========")
  print(tf.reduce_sum(softmax_output, axis=1))

  #Print the below to check that softmax across axis = 0 doesn't sum to one for final check
  #print(tf.reduce_sum(softmax_output, axis=0))
  break;

===== batch number: 0
====== unscaled thing =============
tf.Tensor(
[[ 0.  1.  3. ... 18.  2.  6.]
 [ 0.  5. 18. ...  8. 18.  3.]
 [ 0.  8. 18. ...  3.  1. 18.]
 ...
 [ 0.  6.  8. ...  5.  5.  2.]
 [ 0. 14.  6. ... 14.  5.  8.]
 [ 0. 18.  5. ...  6.  0. 18.]], shape=(128, 200), dtype=float32)

 ===============. softmax thing ===========
tf.Tensor(
[[6.7922040e-10 1.8463124e-09 1.3642507e-08 ... 4.4597588e-02
  5.0187978e-09 2.7401705e-07]
 [4.8667026e-10 7.2228261e-08 3.1954750e-02 ... 1.4507434e-06
  3.1954750e-02 9.7750332e-09]
 [5.7840488e-10 1.7242006e-06 3.7978046e-02 ... 1.1617574e-08
  1.5722674e-09 3.7978046e-02]
 ...
 [5.2054844e-10 2.1000422e-07 1.5517329e-06 ... 7.7256232e-08
  7.7256232e-08 3.8463615e-09]
 [5.1795140e-10 6.2289054e-04 2.0895649e-07 ... 6.2289054e-04
  7.6870798e-08 1.5439913e-06]
 [4.7133214e-10 3.0947652e-02 6.9951888e-08 ... 1.9014895e-07
  4.7133214e-10 3.0947652e-02]], shape=(128, 200), dtype=float32)

======== check whether softmax adds to one for fin